In [ ]:
import numpy as np
import pandas as pd 
import os, sys, gc, time, warnings, pickle, psutil, random

TRAIN_PATH = "../input/ubiquant-parquet/train_low_mem.parquet"
TARGET = "target"

In [ ]:
train = pd.read_parquet(TRAIN_PATH)

def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

train = reduce_memory_usage(train)

gc.collect()

In [ ]:
def getFeatureImportance(df,TARGET,num = -1):
    # correlation 
    corrTrain = df.corr()
    
    # abs 
    corrTrain = np.abs(corrTrain)
    
    # to dataframe 
    corrTarget = corrTrain[[TARGET]]
    
    # sort 
    corrTarget = corrTarget.sort_values(TARGET,ascending=False)
    corrTargetT = corrTarget.T
    
    # delete target
    corrTargetTImprortance = [col for col in corrTargetT.columns if col != TARGET]
    
    # select feature importance top num
    if num != -1:
         selCorrTargetTImprortance = corrTargetTImprortance[:num]
    else:
        selCorrTargetTImprortance = corrTargetTImprortance
        
    result = corrTargetT[selCorrTargetTImprortance]
    
    return result

In [ ]:
result = getFeatureImportance(train,TARGET)
result

In [ ]:
resultT = result.T

df = pd.DataFrame(columns=["index","target"])
df["index"] = resultT.index
df["target"] =  resultT["target"].values

df.to_csv("feature_importance.csv",index=False)

In [ ]:
result.T[:30]